In [1]:
from utils.mypytable import MyPyTable 
import utils.plot_utils as plot_utils
import importlib
import utils.myclassifiers
importlib.reload(utils.myclassifiers)
from utils.myclassifiers import MyDecisionTreeClassifier
from utils.myclassifiers import MyKNeighborsClassifier
from utils.myclassifiers import MyDummyClassifier
from utils.myclassifiers import MyNaiveBayesClassifier

import numpy as np # Added import statement
import utils.myevaluation
importlib.reload(utils.myevaluation)
import utils.myevaluation as myevaluation
from itertools import chain
import utils.myutils as utils
importlib.reload(utils)
importlib.reload(plot_utils)
table = MyPyTable()
file_name = 'Skyserver_SQL4_21_2022 12_41_18 AM.csv'
table.load_from_file('input_folder/' + file_name)

In [2]:
# Removing all the unphysical fluxes
def row_idx_flux(table, col_idxs):
    row_indexes = []
    
    for i in range(len(col_idxs)):
        for j in range(len(table)):
            if j not in row_indexes:
                if (table[j][col_idxs[i]] > (1*10**5)) or (table[j][col_idxs[i]] < 0):
                    row_indexes.append(j)

    return row_indexes

col_idxs = list(range(17, 28 + 1)) # Plus 1 to account for exclusivity (17 start of fluxes)

row_flux =  row_idx_flux(table.data, col_idxs) # Identifies 18 rows with unphysical fluxes
b_len = len(table.data)
table.drop_rows(row_flux)
a_len =len(table.data)
print("Removed instances:", b_len - a_len)

20000
5003


In [3]:
# Data Cleaning 
# Removing all zWarning that are not (0 - OK)
values, counts = utils.get_frequencies(table.data, 14)
print("zwarning values:", values)
print("zWarning counts:", counts) # Removing only 36 instances

def row_indexes_to_drop(table, col_idx):
    row_indexes = []
    row_indexes = [i for i in range(len(table)) if table[i][col_idx] != 0]

    return row_indexes

row_idxs = row_indexes_to_drop(table.data, 14)
print(row_idxs, len(row_idxs))
b_len = len(table.data)
table.drop_rows(row_idxs)
a_len = len(table.data)
print("Removed instances:", b_len - a_len)
# Takes 34s to remove 36 instances

zwarning values: [0.0, 4.0, 16.0, 64.0]
zWarning counts: [4987, 1, 13, 2]
[401, 695, 911, 1240, 1771, 2600, 2791, 2902, 3175, 3322, 3412, 3634, 4181, 4204, 4414, 4935] 16
5003
4987


In [10]:
# Normalizing the dataset 
# def normalization_h_beta(table, col_idxs):
col_idxs = list(range(17, 28 + 1)) # Plus 1 to account for exclusivity (17 start of fluxes)

for i in range(len(col_idxs)):
    for j in range(len(table.data)):
        table.data[j][col_idxs[i]] = table.data[j][col_idxs[i]] / table.data[j][col_idxs[3]] # 3 indicates index of h_beta_flux in col_idxs

In [11]:
# Ensuring erroneous fluxes removed 
summary_table = table.compute_summary_statistics(['p_el', 'p_cw', 'p_acw', 'p_edge', 'p_dk', 'p_mg', 'ra', 'dec', 'z', 'zWarning', 'waveMin', 'plateSN2', 'h_delta_flux', 'h_gamma_flux', 'oiii_4363_flux', 'h_beta_flux', 'oiii_4959_flux', 'oiii_5007_flux', 'hei_5876_flux', 'oi_6300_flux', 'h_alpha_flux', 'nii_6548_flux', 'sii_6717_flux', 'sii_6731_flux'])
summary_table.pretty_print()

attribute                 min           max          mid           avg        median
--------------  -------------  ------------  -----------  ------------  ------------
p_el               0               1            0.5          0.403618      0.422
p_cw               0               1            0.5          0.109711      0.029
p_acw              0               1            0.5          0.114025      0.033
p_edge             0               1            0.5          0.259218      0.2
p_dk               0               0.857        0.4285       0.080292      0.058
p_mg               0               0.926        0.463        0.0330559     0
ra                 0.0421753     359.853      179.947      200.028       187.864
dec               -3.74089        66.1556      31.2074       5.3555       -0.264008
z                  0.0100996       0.345784     0.177942     0.0870138     0.0840691
zWarning           0               0            0            0             0
waveMin         3785.3 

## Next Steps 
1. Remove particularly weak emission lines (need to talk to Dr. Aver about this one)
1. Normalizaion with h_beta_flux 